In [6]:
!pip install pandas
!pip install matplotlib

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [74]:

def compute_mean_std(attack,combined_df ) -> None:
    attack_df = combined_df[combined_df['Attack Type'] == attack]
    
    # Store epsilons in a dictionary
    epsilons = {
        'eps1': attack_df[attack_df['Epsilon'] == 0.00],
        'eps2': attack_df[attack_df['Epsilon'] == 0.01],
        'eps3': attack_df[attack_df['Epsilon'] == 0.10],
        'eps4': attack_df[attack_df['Epsilon'] == 0.15],
    }
    
    for name, df in epsilons.items():
        # Drop non-metric columns
        metrics_df = df.drop(columns=['Run ID', 'Epsilon', 'Attack Type','loss','recall','roc_auc', 'precision','pr_auc'], errors='ignore')
        
        # Calculate mean and std
        mean_series = (metrics_df.mean() * 100).round(2)
        std_series = (metrics_df.std() * 100).round(2)
        
        
        # Combine into a single DataFrame
        result_df = pd.DataFrame([mean_series, std_series], index=['mean', 'std'])
        
        print(f"\n{name} (Epsilon = {df['Epsilon'].iloc[0]}):")
        print(result_df)


def load_combine(csv_files):
    columns_to_analyze = [
        'accuracy', 'loss', 'precision', 'recall', 'f1', 'fpr', 'fnr', 'roc_auc', 'pr_auc'
    ]
    ndfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        ndfs.append(df)
    combined_df = pd.concat(ndfs, ignore_index=True)  
    # print(combined_df.head())
    # print(len(combined_df))
    return combined_df

In [75]:
csv_files = [
    'qmlp-az-run_run1-layer2_Amplitude_all_attacks.csv',
    'qmlp-az-run_run2-layer2_Amplitude_all_attacks.csv',
    'qmlp-az-run_run3-layer2_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     40.11  38.74  2.73  60.48
std       0.11   0.15  0.01   0.26

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     13.68  12.92  3.94  87.21
std       1.41   0.34  0.06   1.53

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      0.73  0.36  4.53  99.27
std       0.16  0.17  0.01   0.02

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean      0.35  0.19  4.54  99.67
std       0.20  0.18  0.01   0.18
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     40.11  38.74  2.73  60.48
std       0.11   0.15  0.01   0.26

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     13.12  12.33  3.96  87.73
std       1.50   0.47  0.07   1.57

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      0.19  0.05  4.54  99.82
std       0.12  0.03  0.02   0.08

eps4 (Epsilon = 0.15):
      accuracy   f1   fpr    fnr
mean      0.01  0.0  4.54  99.99
std       0

In [76]:
csv_files = [
    'qmlp-az-run_run1-layer10_Amplitude_all_attacks.csv',
    'qmlp-az-run_run2-layer10_Amplitude_all_attacks.csv',
    'qmlp-az-run_run3-layer10_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     55.04  53.88  2.05  45.83
std       1.36   0.40  0.06   1.27

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     15.76  16.48  3.84  84.74
std       0.26   1.38  0.01   0.47

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      0.57  0.39  4.54  99.46
std       0.32  0.21  0.01   0.31

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean      0.11  0.06  4.56  99.90
std       0.07  0.02  0.00   0.07
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     55.04  53.88  2.05  45.83
std       1.36   0.40  0.06   1.27

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     14.18  14.94  3.92  86.22
std       0.45   1.72  0.02   0.55

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      0.07  0.04  4.55  99.93
std       0.06  0.04  0.00   0.07

eps4 (Epsilon = 0.15):
      accuracy   f1   fpr    fnr
mean       0.0  0.0  4.55  100.0
std        

In [77]:
csv_files = [
    'qmlp-az-run_run1-layer50_Amplitude_all_attacks.csv',
    'qmlp-az-run_run2-layer50_Amplitude_all_attacks.csv',
    'qmlp-az-run_run3-layer50_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     66.90  65.30  1.51  34.27
std       0.66   0.65  0.03   0.46


IndexError: single positional indexer is out-of-bounds

In [78]:
csv_files = [
    'qmlp-az-run_run1-layer2_Angle_all_attacks.csv',
    'qmlp-az-run_run2-layer2_Angle_all_attacks.csv',
    'qmlp-az-run_run3-layer2_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     49.77  48.41  2.29  50.88
std       0.16   0.37  0.01   0.17

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     19.53  19.68  3.68  82.42
std       0.83   1.07  0.04   0.75

eps3 (Epsilon = 0.1):
      accuracy     f1   fpr    fnr
mean     15.77  15.31  3.85  85.84
std       0.66   0.99  0.03   0.62

eps4 (Epsilon = 0.15):
      accuracy     f1   fpr    fnr
mean     13.93  13.19  3.93  87.50
std       0.78   1.14  0.04   0.71
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     49.77  48.41  2.29  50.88
std       0.16   0.37  0.01   0.17

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     19.44  19.78  3.68  82.49
std       0.83   0.97  0.04   0.74

eps3 (Epsilon = 0.1):
      accuracy     f1   fpr    fnr
mean     15.41  15.11  3.86  86.17
std       0.85   0.58  0.04   0.78

eps4 (Epsilon = 0.15):
      accuracy     f1   fpr    fnr
mean     13.13  12.56  3.97  88.2

In [66]:
csv_files = [
    'qmlp-az-run_run1-layer10_Angle_Angle_all_attacks.csv',
    'qmlp-az-run_run2-layer10_Angle_Angle_all_attacks.csv',
    'qmlp-az-run_run3-layer10_Angle_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     54.88  53.18  2.05  45.89
std       0.82   0.97  0.04   0.96

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean     10.83  9.54  4.07  90.22
std       0.56  0.56  0.02   0.50
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     54.88  53.18  2.05  45.89
std       0.82   0.97  0.04   0.96

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean      9.52  7.70  4.13  91.45
std       0.26  0.82  0.01   0.21


In [67]:
# qmlp-mnist-run_run1-layer2_Angle_all_attacks.csv
csv_files = [
    'qmlp-mnist-run_run1-layer2_Angle_all_attacks.csv',
    'qmlp-mnist-run_run2-layer2_Angle_all_attacks.csv',
    'qmlp-mnist-run_run3-layer2_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     65.93  65.05  3.79  34.68
std       1.04   1.03  0.12   1.12

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      1.29  0.98  10.91  98.58
std       0.24  0.17   0.02   0.28
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     65.93  65.05  3.79  34.68
std       1.04   1.03  0.12   1.12

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      0.50  0.21  10.99  99.46
std       0.03  0.04   0.01   0.03


In [82]:
csv_files = [
    'qmlp-mnist-run_run1-layer10_Angle_all_attacks.csv',
    'qmlp-mnist-run_run2-layer10_Angle_all_attacks.csv',
    'qmlp-mnist-run_run3-layer10_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     71.72  71.16  3.14  28.80
std       1.80   1.81  0.20   1.86

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     27.12  34.03  8.04  71.83
std       3.94   2.75  0.43   3.83

eps3 (Epsilon = 0.1):
      accuracy    f1    fpr    fnr
mean      3.81  3.73  10.63  95.99
std       0.54  0.39   0.08   0.64

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      1.44  1.29  10.90  98.48
std       0.42  0.47   0.06   0.47
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     71.72  71.16  3.14  28.80
std       1.80   1.81  0.20   1.86

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     26.27  33.17  8.13  72.69
std       3.62   2.37  0.39   3.51

eps3 (Epsilon = 0.1):
      accuracy    f1    fpr    fnr
mean      2.62  1.87  10.75  97.21
std       0.61  0.22   0.08   0.67

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      0.64  0.34  10.98  99.3

In [69]:
csv_files = [
    'qmlp-mnist-run_run1-layer50_Angle_all_attacks.csv',
    'qmlp-mnist-run_run2-layer50_Angle_all_attacks.csv',
    'qmlp-mnist-run_run3-layer50_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     76.89  76.61  2.57  23.46
std       0.43   0.35  0.05   0.43

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      3.64  3.80  10.66  96.23
std       0.53  0.47   0.06   0.55
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     76.89  76.61  2.57  23.46
std       0.43   0.35  0.05   0.43

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      0.73  0.65  10.97  99.24
std       0.12  0.14   0.02   0.12


In [81]:
csv_files = [
    'qmlp-mnist-run_run1-layer50_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run2-layer50_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run3-layer50_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr   fnr
mean     92.64  92.56  0.82  7.44
std       0.49   0.51  0.05  0.52

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     27.26  32.58  8.08  72.59
std       2.45   2.82  0.28   2.42

eps3 (Epsilon = 0.1):
      accuracy    f1    fpr    fnr
mean      0.02  0.01  11.10  99.98
std       0.02  0.01   0.01   0.02

eps4 (Epsilon = 0.15):
      accuracy   f1    fpr    fnr
mean       0.0  0.0  11.10  100.0
std        0.0  0.0   0.01    0.0
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr   fnr
mean     92.64  92.56  0.82  7.44
std       0.49   0.51  0.05  0.52

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     21.92  26.13  8.67  77.90
std       1.72   2.17  0.20   1.71

eps3 (Epsilon = 0.1):
      accuracy   f1    fpr    fnr
mean       0.0  0.0  11.11  100.0
std        0.0  0.0   0.01    0.0

eps4 (Epsilon = 0.15):
      accuracy   f1    fpr    fnr
mean       0.0  0.0  11.11  100.0
std        0

In [80]:
csv_files = [
    'qmlp-mnist-run_run1-layer10_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run2-layer10_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run3-layer10_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     79.82  79.56  2.24  20.47
std       1.03   1.10  0.11   1.07

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     22.27  24.98  8.63  77.61
std       3.57   5.40  0.39   3.56

eps3 (Epsilon = 0.1):
      accuracy    f1    fpr    fnr
mean      0.98  0.81  10.99  99.02
std       0.82  0.60   0.09   0.81

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      0.10  0.07  11.09  99.90
std       0.15  0.10   0.01   0.15
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     79.82  79.56  2.24  20.47
std       1.03   1.10  0.11   1.07

eps2 (Epsilon = 0.01):
      accuracy     f1   fpr    fnr
mean     19.20  21.38  8.97  80.69
std       3.37   5.19  0.37   3.37

eps3 (Epsilon = 0.1):
      accuracy    f1    fpr    fnr
mean      0.10  0.06  11.09  99.90
std       0.15  0.09   0.01   0.15

eps4 (Epsilon = 0.15):
      accuracy   f1    fpr    fnr
mean       0.0  0.0  11.10  100.0


In [72]:
csv_files = [
    'qmlp-mnist-run_run1-layer2_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run2-layer2_Amplitude_all_attacks.csv',
    'qmlp-mnist-run_run3-layer2_Amplitude_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     65.93  65.05  3.79  34.68
std       1.04   1.03  0.12   1.12

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      1.29  0.98  10.91  98.58
std       0.24  0.17   0.02   0.28
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     65.93  65.05  3.79  34.68
std       1.04   1.03  0.12   1.12

eps4 (Epsilon = 0.15):
      accuracy    f1    fpr    fnr
mean      0.48  0.27  10.99  99.47
std       0.09  0.11   0.01   0.09


In [79]:
csv_files = [
    'qmlp-az-run_run1-layer50_Angle_Angle_all_attacks.csv',
    'qmlp-az-run_run2-layer50_Angle_Angle_all_attacks.csv',
    'qmlp-az-run_run3-layer50_Angle_Angle_all_attacks.csv'
]
df = load_combine(csv_files)
print('FGSM')
compute_mean_std('fgsm',df)
print('PGD')
compute_mean_std('pgd',df)

FGSM

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     32.38  29.45  3.09  69.58
std       0.90   1.48  0.04   0.88

eps2 (Epsilon = 0.01):
      accuracy    f1   fpr    fnr
mean      5.17  3.89  4.33  95.28
std       0.47  0.55  0.02   0.47

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      5.01  4.15  4.34  95.41
std       0.28  0.32  0.01   0.31

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean      5.04  4.06  4.33  95.37
std       0.21  0.45  0.01   0.23
PGD

eps1 (Epsilon = 0.0):
      accuracy     f1   fpr    fnr
mean     32.38  29.45  3.09  69.58
std       0.90   1.48  0.04   0.88

eps2 (Epsilon = 0.01):
      accuracy    f1   fpr    fnr
mean      4.20  3.50  4.37  96.19
std       0.32  0.25  0.02   0.31

eps3 (Epsilon = 0.1):
      accuracy    f1   fpr    fnr
mean      1.05  0.69  4.52  99.05
std       0.05  0.01  0.00   0.04

eps4 (Epsilon = 0.15):
      accuracy    f1   fpr    fnr
mean      0.57  0.38  4.54  99.48
std       0.10 